## Setting up

In [1]:
#Import packages

import numpy as np
import pandas as pd 
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split


data = pd.read_csv("rao_imr90.tads.boundary.4mer.features.csv")
#data

C:\Users\leyen\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251

In [2]:
#Take out the unneeded row
i = data[(data.AAAA == 'AAAA')].index
data = data.drop(i)

In [3]:
#Indicate features and class label
x = data.iloc[:,1:-1].apply(pd.to_numeric,errors='coerce')
y = data["class"].astype('int')

In [4]:
#Split data into training (80%) and testing (testing)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20)

In [5]:
#Build the logistic regression model
model = LogisticRegression(max_iter=10000)
model.fit(x_train, y_train)

LogisticRegression(max_iter=10000)

In [6]:
#Check y prediction
y_pred = model.predict(x_test)

y_pred

array([1, 0, 0, ..., 0, 0, 0])

In [7]:
param_grid = [    
    {'penalty' : ['l2'],
    'C' : ( 0.0001, 0.001, 0.01, 0.1, 1.0),
    'solver' : ['lbfgs','newton-cg','liblinear','sag','saga'],
    'max_iter' : [25,50]
    } ]

In [8]:
# set up grid search meta-estimator
clf = GridSearchCV(model, param_grid, cv=10, n_jobs=-1)

# train the grid search meta-estimator to find the best model
best_model = clf.fit(x, y)

C:\Users\leyen\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [9]:
#Print out the the best parameter and score
params = clf.best_params_
print("Best Parameters: ", params)

score = clf.best_score_
print("Best Score: ", score)

Best Parameters:  {'C': 0.001, 'max_iter': 25, 'penalty': 'l2', 'solver': 'lbfgs'}
Best Score:  0.6185695538057743


## Accuracy, AUC (area under the curve) score, Precision, Recall, F1 score

In [12]:
#Accuracy, Precision, Recall, F1 score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

print("Accuracy: ", accuracy_score(y_test, y_pred))
print("Precision: ", precision_score(y_test, y_pred))
print("Recall: ", recall_score(y_test, y_pred))
print("F1 score: ", f1_score(y_test, y_pred))

Accuracy:  0.610236220472441
Precision:  0.5998645903859174
Recall:  0.5974376264329063
F1 score:  0.5986486486486486


In [13]:
from sklearn.model_selection import cross_val_score

# AUC 
clf_labels = ['Logistic Regression']
for clf, label in zip([model], clf_labels):
    scores = cross_val_score(model, x_train, y_train, cv=10, scoring='roc_auc')

In [14]:
print("ROC AUC: ", scores.mean())

ROC AUC:  0.6583627556451992


## Save the trained model to a file

In [15]:
import pickle 
filename = 'rao_imr.pickle'
pickle.dump(model, open(filename, 'wb'))